### Notebook to exemplify the Hedge_latencies. 
This notebook outlines latencies specific to our specific algorithm or autospreader. They depend on our TT server. <br>
The definitions of the exchanges calculated here are inserted below
-   Hedge Latency <br> 
    The time taken by our trading machine to send a hedge order after we receive a fill on the active leg of an autospreader.  It can be computed in this notebook by passing a csv to the process\_csv function and then calling _hedge\_latency function using the following: _hedge_latency(orders_df, ASELatency._hedge_latency_single_leg,"hedge_latency")<br>The parameters are: <br>
    - orders_df: The dataframe of the AuditTrail.
    - "ASELatency._hedge_latency_single_leg: Function specifying what latency we compute, determines if we compute latency with acknowledgement or not. 
    - "hedge_latency": Specifies that we are considering hedge latency without acknowledgement. 
-   Hedge Latency with acknowledgement <br>
    The time between when we receive a fill message, and the time we are acknowledged our new hedge order has arrived in the exchange. It can be computed in the notebook by passing a csv to the process\_csv function and then calling \_hedge\_latency function using the following:  _hedge_latency(orders_df, ASELatency._hedge_latency_with_ack_time_single_leg, "hedge_latency_with_ack")<br> The parameters are: <br>
    - orders_df: The dataframe of the AuditTrail.
    - "ASELatency._hedge_latency_with_ack_time_single_leg: Function specifying what latency we compute, determines if we compute latency with acknowledgement or not. 
    - "hedge_latency_with_ack": Specifies that we are considering hedge latency with acknowledgement. 
    <br>



In [9]:
# imports
import numpy as np
import pandas as pd
from latency_config import (
    ATColumns,
    ETypes,
    SynchMode,
    QuotesColumns,
    TradesColumns,
    QUOTE_ORDERS,
    ORDER_TYPE_TO_SIDE,
    AT_TABLE_PKEYS,
)
from analysis_metrics.base.base_latency import BaseLatency
from analysis_metrics.base.ase_latency import ASELatency
import datetime as dt
from datetime import timezone
from audit_trail_manipulation import ATManipulation
import utils

In [10]:
def requote_latency(
        cls,
        liquid_leg_quotes: pd.DataFrame,
        illiquid_leg_quotes: pd.DataFrame,
        threshold=None,
    ):
        """
        Computes the time it takes to our logic to send a new quote after a market update. The market updates are our own quotes.
        We are assuming we receive the acknowledge roughly at the same time we receive the market update, so we use the field `ATColumns.TIME_SENT`
        of our own quotes.

        Parameters
        ----------
        liquid_leg_quotes : pd.DataFrame
            Quotes that we send after reacting to the market
        illiquid_leg_quotes : pd.DataFrame
            Quotes we react to
        threshold : int
            Offsets larger than this will be dropped and won't be used, neither for statistics nor for plotting etc.
        Returns
        -------
        pd.DataFrame
            Dataframe with "requote_latency", and timestamps of the illiquid and liquid legs.
        """
        # mask_new = liquid_leg_quotes[ATColumns.EXECUTION_TYPE] == ETypes.NEW
        requote_time = liquid_leg_quotes[ATColumns.EXCHANGE_TIME].copy()
        # requote_time.loc[mask_new] = liquid_leg_quotes.loc[
        #     mask_new, ATColumns.ORIGINAL_TIME
        # ]
        requote_time = requote_time.apply(lambda timestamp: timestamp.floor("ms"))
        received_time = illiquid_leg_quotes[ATColumns.TIME_SENT].apply(
            lambda timestamp: timestamp.floor("ms")
        )
        result = pd.DataFrame(
            data={
                "requote_latency": BaseLatency.to_millisec(
                    requote_time - received_time.values
                ).values,
                "illiquid_dtime": illiquid_leg_quotes.index,
                "illiquid_row_id": illiquid_leg_quotes[ATColumns.DF_ROW_ID].values,
                "liquid_dtime": liquid_leg_quotes.index,
                "liquid_row_id": liquid_leg_quotes[ATColumns.DF_ROW_ID].values,
            },
            index=liquid_leg_quotes.index,
        )
        if threshold is not None:
            return result[result["requote_latency"] <= threshold]
        return result

In [11]:
# Cell to compute the hedge latency
def hedge_latency_single(  # TODO: fix for exchange-traded calendar/flies
        
        tt_parent_id: str,
        ase_name: str,
        latency_name: str,
        df_group: pd.DataFrame,
        latency_fn,
    ):  # Already grouped by tt_parent_id
        """
        Performs aggregation of DataFrame rows that have the same "tt_parent_id".
        Parameters
        ----------
        tt_parent_id : str
            Order id of the spread for which we want to compute the latencies.
        spread_id : str
            Name of the parent instrument
        latency_name : str
            Type of latency measure we are computing.
        df_group : pd.DataFrame
            Spread legs
        latency_fn : (str, pd.DataFrame, pd.DataFrame) -> pd.DataFrame
            Function that take latency description, active legs dataframe and hedge legs dataframe to compute latency


        Returns
        -------
        pd.DataFrame
            DataFrame with matched new orders and fills
        """
        # separate orders in TRADES and NEWS
        legs = df_group[ATColumns.INSTRUMENT].unique()
        fills = df_group[df_group[ATColumns.EXECUTION_TYPE] == ETypes.TRADE]
        new_orders = df_group[df_group[ATColumns.EXECUTION_TYPE] == ETypes.NEW]
        if fills.empty:  # Exclude groups without trades
            return None
        # assume the first traded instrument is the active leg
        active_leg = fills[ATColumns.INSTRUMENT].iloc[0]
        active_leg_fills = fills[fills[ATColumns.INSTRUMENT] == active_leg]
        # all the new orders that don't involve the active leg are hedge orders
        hedge_new_orders = new_orders[new_orders[ATColumns.INSTRUMENT] != active_leg]
        # for each hedge instrument we have a dataframe with the new orders of that instrument
        hedge_legs_new_orders = [
            group
            for _, group in hedge_new_orders.groupby(
                ATColumns.INSTRUMENT
            )  # the version with [ATColumns.INSTRUMENT] gives a warning in pandas
        ]
        if hedge_new_orders.empty:  # Exclude groups without hedge orders
            LOG.warn(f"{tt_parent_id=} does not have any hedge orders in period")
            return None
        # all the dataframes in the hedge_legs_new_orders list must have the same size as the dataframe with the active leg fills, i.e. the active leg
        # and each hedge leg must have been traded with the same number of new orders
        if any(
            len(hedge_new_orders) != len(active_leg_fills)
            for hedge_new_orders in hedge_legs_new_orders
        ):
            # NOTE: Here we simply import ASE module and then call _handle_unmatched_orders through that. we have removed 
            leg_latencies = ASELatency._handle_unmatched_orders(
                active_leg_fills,
                df_group,
                tt_parent_id,
                hedge_legs_new_orders,
                latency_fn,
                latency_name,
                legs,
            )
            if leg_latencies == None:
                LOG.warn(
                    f"WARN: {tt_parent_id=} has unmatching active fills and hedge orders. Legs={legs}"
                )
                return None
        else:
            # if the times of the orders are too far apart we are doing a mismatch
            # NOTE: We have imported ASELatency. 
            if ASELatency._distant_orders(
                hedge_legs_new_orders, active_leg_fills, threshold=-10
            ):
                return None
            # creates a list with dataframes each one containing a measurement of latency (by the latency function)
            leg_latencies = [
                latency_fn(latency_name, ase_name, active_leg_fills, leg_orders)
                for leg_orders in hedge_legs_new_orders
            ]
        # concatenates the dataframes (on the index axis). In theory you can get a difference latency for each hedge leg
        return ASELatency._combine_leg_latencies(leg_latencies, latency_name=latency_name)


In [12]:
def _hedge_latency(df: pd.DataFrame, latency_fn, latency_name):
        """
        Parameters
        ----------
        df : pd.DataFrame
            AuditTrail data
        latency_fn : (str, pd.DataFrame, pd.DataFrame) -> pd.DataFrame
            Function measuring the latency (depends of course on the kind of latency being measured), this determines whether it is with or without acknowledgement
        latency_name : str
            The kind of latency being measured

        Returns
        -------
        pd.DataFrame
            Hedge latencies
        """
        hedge_latencies = df.groupby([ATColumns.TT_PARENT_ID]).apply(
            lambda df_group: hedge_latency_single(
                tt_parent_id=df_group.name,
                ase_name=ASELatency._get_ase_name_from_order_id(df, df_group.name),
                latency_name=latency_name,
                df_group=df_group,
                latency_fn=latency_fn,
            )
        )
        if hedge_latencies.empty:
            hedge_latencies[latency_name] = None

        hedge_latencies = hedge_latencies.reset_index(level=[0]).dropna(
            subset=latency_name
        )
        return hedge_latencies

In [13]:
# Read Comma separated file. 
def process_csv(csv_name):
    orders_df = pd.read_csv(csv_name, index_col="dtime")
    orders_df[ATColumns.TIME_SENT]=orders_df.index
    orders_df = BaseLatency.prepare_loaded(orders_df)
    return orders_df
orders_df = process_csv("raw_df.csv")

In [14]:
hedge_latencies = _hedge_latency(orders_df, ASELatency._hedge_latency_single_leg,"hedge_latency")
print(hedge_latencies['hedge_latency'])
hedge_latencies_with_ack=_hedge_latency(
            orders_df, ASELatency._hedge_latency_with_ack_time_single_leg, "hedge_latency_with_ack"
        )
print(hedge_latencies_with_ack['hedge_latency_with_ack'])

dtime
2024-02-02 13:30:01.091000+00:00    0.0
2024-02-02 13:30:01.092000+00:00    0.0
2024-02-02 13:30:01.092000+00:00    0.0
2024-02-02 13:30:01.092000+00:00    0.0
2024-02-02 13:30:01.092000+00:00    0.0
Name: hedge_latency, dtype: float64
dtime
2024-02-02 13:30:01.091000+00:00    12.0
2024-02-02 13:30:01.092000+00:00    13.0
2024-02-02 13:30:01.092000+00:00    13.0
2024-02-02 13:30:01.092000+00:00    13.0
2024-02-02 13:30:01.092000+00:00    13.0
Name: hedge_latency_with_ack, dtype: float64


C:\Users\ctardon\AppData\Local\Temp\ipykernel_13772\2755470372.py:17: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  hedge_latencies = df.groupby([ATColumns.TT_PARENT_ID]).apply(
C:\Users\ctardon\AppData\Local\Temp\ipykernel_13772\2755470372.py:17: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  hedge_latencies = df.groupby([ATColumns.TT_PARENT_ID]).apply(
